In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import myutils as u
import math
import datetime
import warnings
import pandas.errors
warnings.simplefilter(action='ignore', category=Warning)

In [2]:
ALL_TC_SITES = u.import_data("SITES")
ALL_TC = u.import_data("TRAFFIC_COUNTS")
ALL_OTP = u.import_data("ON_TIME")
ALL_STOPS = u.import_data("STOPS")
ALL_CONSTR = u.import_data("LANE_CLOSURE")
ALL_ROADS = u.import_data("ROAD_NETWORK")

# Consts
MIN = pd.to_timedelta("1 min")
DAY = pd.to_timedelta("1 day")
SITES_NO = {'McPhillips': 0,'Henderson': 1,'Pembina': 2,'Inkster': 3,'Nichol': 4,'Lagimodiere': 5,'Disraeli': 6,'Marion': 7} 

In [3]:
date = pd.to_datetime("2021-08-23")
distance=500
time = pd.to_datetime("16:00:00").time()

In [4]:
def get_construction_sites(date,time=None):
    if time == None:
        return ALL_CONSTR[(ALL_CONSTR["Date Closed - From"] <= date) & (ALL_CONSTR["Date Closed - To"] > date)]
    elif isinstance(time,datetime.time):
        return ALL_CONSTR[(ALL_CONSTR["Date Closed - From"] <= date) & (ALL_CONSTR["Date Closed - To"] > date)
                & (((ALL_CONSTR["Time Closed - From"] < time) & (ALL_CONSTR["Time Closed - To"] == pd.to_datetime("00:00:00").time())) 
                | ((ALL_CONSTR["Time Closed - From"] <= time) & (ALL_CONSTR["Time Closed - To"] > time)))]  

get_construction_sites(date,time=pd.to_datetime("16:00:00").time())

,Street,At,Boundaries,Direction,Date Closed - From,Date Closed - To,Time Closed - From,Time Closed - To,Complete Closure,Lat,Long,Geometry,Num_Segments,Num_Points
0,Bryce,River,"[River Av, End]","[Eastbound, Westbound]",2021-08-16,2021-10-16,00:00:00,00:00:00,No,49.879699,-97.142781,"[[(49.879153816388, -97.142387643857), (49.880...",1,3
1,Lilac,Carter,"[Carter Av, Weatherdon Av]",[Southbound],2021-06-14,2021-10-29,00:00:00,00:00:00,No,49.862454,-97.151445,"[[(49.862119196838, -97.151173092203), (49.862...",1,5
2,Princess,Pacific,"[Pacific Av, Rupert Av]",[Southbound],2020-08-14,2021-10-16,00:00:00,00:00:00,No,49.901786,-97.139766,"[[(49.90158994935, -97.139915854251), (49.9019...",1,3
3,Henry,Austin,"[Austin St, End]",[Westbound],2021-05-10,2021-11-30,00:00:00,23:00:00,No,49.903340,-97.134152,"[[(49.903503694415, -97.13466284795), (49.9033...",1,4
4,Grandin,Tache,"[Tache Av, St Joseph St]",[Eastbound],2021-05-31,2022-01-13,00:00:00,00:00:00,No,49.896013,-97.126315,"[[(49.895678857718, -97.127616387474), (49.896...",1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,Munroe,Henderson,"[Henderson Hw, Golspie St]",[Westbound],2021-05-03,2021-10-08,00:00:00,00:00:00,Yes,49.919450,-97.103062,"[[(49.92159361799, -97.109515392943), (49.9214...",1,7
403,Stafford,Dudley,"[Dudley Av, Warsaw Av]",[Northbound],2021-08-17,2021-10-15,00:00:00,00:00:00,No,49.865149,-97.159055,"[[(49.8637858332, -97.157961213665), (49.86386...",1,17
404,Bishop Grandin,Lagimodiere,"[Lagimodiere Bv, De La Seigneurie Bv]",[Eastbound],2021-08-07,2021-10-15,00:00:00,00:00:00,No,49.840370,-97.053552,"[[(49.840625586473, -97.057348732447), (49.840...",1,15
405,Winterton,Roch,"[Roch St, Watt St]",[Southbound],2021-08-19,2021-10-08,00:00:00,00:00:00,No,49.917711,-97.100369,"[[(49.918363079885, -97.102331187677), (49.917...",1,3


In [8]:
def get_stops_distances(constr_index, distance=500,key=None):
    site_coords = ALL_CONSTR.loc[constr_index,["Lat","Long"]].values
    stop_coords = ALL_STOPS[["Lat","Long"]].values
    distances = u.distance_within(site_coords,stop_coords,distance,key=key)
    return distances 

def get_stops_nearby(constr_index, distance=500):
    stops_index = get_stops_distances(constr_index, distance, "index")
    return ALL_STOPS.iloc[stops_index]

get_stops_nearby(325)

[49.8621839404672 -97.2001804295113]


,Stop Number,Stop Name,Lat,Long,Direction,Street,At
4350,60381,Westbound Corydon at Lindsay,49.863772,-97.193756,Westbound,Corydon,Lindsay
4351,60382,Eastbound Corydon at Lindsay,49.863561,-97.194056,Eastbound,Corydon,Lindsay
4352,60383,Westbound Corydon at Lanark,49.863925,-97.197758,Westbound,Corydon,Lanark
4353,60384,Eastbound Corydon at Lanark,49.863726,-97.198280,Eastbound,Corydon,Lanark
4354,60385,Westbound Corydon at Centennial,49.864058,-97.201107,Westbound,Corydon,Centennial
4355,60386,Eastbound Corydon at Centennial,49.863856,-97.201406,Eastbound,Corydon,Centennial
4356,60387,Westbound Corydon at Lockwood,49.864120,-97.202774,Westbound,Corydon,Lockwood
4357,60388,Eastbound Corydon at Lockwood,49.863919,-97.203171,Eastbound,Corydon,Lockwood
4397,60429,Southbound Kenaston at Carpathia South,49.860570,-97.206496,Southbound,Kenaston,Carpathia South
4398,60430,Northbound Kenaston at Carpathia South,49.861024,-97.206131,Northbound,Kenaston,Carpathia South
